## Objectives:

Perform an Exploratory Data Analysis and determine training labels.

<br>

- Check for missing values and outliers
- Check for correlation between features
- Check for target distribution
- Check for class imbalance
- Split the training, test and validation sets

<br>

> API URL: https://api.spacexdata.com/v4/

### Setting Up

In [7]:
%pip install -Uqq sqlalchemy==1.3.9 ipython-sql sql

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import datetime
import requests
import urllib3
import sqlite3
import json
import time
import csv

connection = sqlite3.connect('data/data.db')
cur = connection.cursor()

%load_ext sql
%sql sqlite:///data/data.db

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", connection, if_exists='replace', index=False, method="multi")

101

In [9]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///data/data.db
Done.


[]

In [20]:
%sql select distinct(Launch_Site) from SPACEXTABLE

 * sqlite:///data/data.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [23]:
%sql select distinct(Launch_Site) from SPACEXTABLE where Launch_Site like 'CCA%'

 * sqlite:///data/data.db
Done.


Launch_Site
CCAFS LC-40
CCAFS SLC-40


In [28]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTABLE where CUSTOMER = 'NASA (CRS)'

 * sqlite:///data/data.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


In [30]:
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTABLE where BOOSTER_VERSION = 'F9 v1.1'

 * sqlite:///data/data.db
Done.


avg(PAYLOAD_MASS__KG_)
2928.4


In [33]:
%sql select min(Date) from SPACEXTABLE where LANDING_OUTCOME = 'Success (ground pad)'

 * sqlite:///data/data.db
Done.


min(Date)
2015-12-22


In [35]:
%sql select distinct(BOOSTER_VERSION) from SPACEXTABLE where LANDING_OUTCOME = 'Success (drone ship)' and PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ < 6000

 * sqlite:///data/data.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [36]:
%sql select count(MISSION_OUTCOME) from SPACEXTABLE where MISSION_OUTCOME = 'Success'

 * sqlite:///data/data.db
Done.


count(MISSION_OUTCOME)
98


In [37]:
%sql select distinct(BOOSTER_VERSION) from SPACEXTABLE where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)

 * sqlite:///data/data.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [40]:
%sql select substr(Date, 6,2) as month, LANDING_OUTCOME, BOOSTER_VERSION, LAUNCH_SITE from SPACEXTABLE where substr(Date,0,5)='2015' and LANDING_OUTCOME like 'Failure%' and BOOSTER_VERSION is not null and LAUNCH_SITE is not null

 * sqlite:///data/data.db
Done.


month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [42]:
%sql select substr(Date, 0, 11) as date, LANDING_OUTCOME, count(LANDING_OUTCOME) as count from SPACEXTABLE where substr(Date, 0, 11) between '2010-06-04' and '2017-03-20' group by LANDING_OUTCOME order by count desc

 * sqlite:///data/data.db
Done.


date,Landing_Outcome,count
2012-05-22,No attempt,10
2016-04-08,Success (drone ship),5
2015-01-10,Failure (drone ship),5
2015-12-22,Success (ground pad),3
2014-04-18,Controlled (ocean),3
2013-09-29,Uncontrolled (ocean),2
2010-06-04,Failure (parachute),2
2015-06-28,Precluded (drone ship),1
